In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm
import sys
import os
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from pathlib import Path
from torchvision.transforms import v2
from tqdm.auto import tqdm
from timeit import default_timer as timer

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(42)
torch.cuda.manual_seed(42)
device

### Base model_V0

In [ ]:
%load_ext autoreload
%autoreload 2

nb_dir = os.path.split(os.getcwd())[0]
nb_dir
print(nb_dir)

if nb_dir not in sys.path:
    print("Nb dir is not in system path")
    sys.path.append(nb_dir)
else:
    print("Nb dir already exists in sys path")

### Data Transforms

In [ ]:
from utils.util import plot_transformed_image

root_image_path = Path().cwd().parent / "data" / "Fast_Food_Classification_V2" / "Train"

test_images_path = [f for f in root_image_path.rglob("*.jpeg")]

data_transform_v0 = v2.Compose([
    # resize image
    v2.Resize(size=(64, 64)),

    # flip images
    v2.RandomHorizontalFlip(p=0.5),

    # turn image to torch tensor
    v2.ToTensor()
])

plot_transformed_image(image_path=test_images_path,
                       transform=data_transform_v0,
                       seed=None,
                       n=2)

### DataLoader & ImageFolder

In [ ]:
# We skipped the train/test/validation split since the dataset had already been filtered

root_train_folder_path = Path().cwd().parent / "data" / "Fast_Food_Classification_V2" / "Train"
root_validation_folder_path = Path().cwd().parent / "data" / "Fast_Food_Classification_V2" / "Valid"
root_test_folder_path = Path().cwd().parent / "data" / "Fast_Food_Classification_V2" / "Test"

train_dataset = ImageFolder(root=root_train_folder_path,
                            transform=data_transform_v0,
                            target_transform=None,  
                            allow_empty=True)

validation_dataset = ImageFolder(root=root_validation_folder_path,
                            transform=data_transform_v0,
                            target_transform=None,
                            allow_empty=True)

test_dataset = ImageFolder(root=root_test_folder_path,
                            transform=data_transform_v0,
                            target_transform=None,
                            allow_empty=True)

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=32,
                              num_workers=1,
                              shuffle=True)

validation_dataloader = DataLoader(dataset=validation_dataset,
                              batch_size=32,
                              num_workers=1,
                              shuffle=False)

test_dataloader = DataLoader(dataset=test_dataset,
                              batch_size=32,
                              num_workers=1,
                              shuffle=False)

### Training The Model

In [ ]:
from classes.model_0 import Cnn_v0

torch.manual_seed(42)
torch.cuda.manual_seed(42)

model_0 = Cnn_v0(input_shape=3,
                 hidden_units=10,
                 output_shape=len(train_dataset.classes)).to(device)

optimizer = torch.optim.Adam(params=model_0.parameters(),
                             lr=0.001)

loss_fn = torch.nn.CrossEntropyLoss()

# model_0.conv_block_2(model_0.conv_block_1(torch.rand(32, 3, 64, 64).to(device))).shape

In [ ]:
from utils.model_utils import train_step, valid_step, test_step, train
from classes.helpers import EarlyStopping

NUM_EPOCHS = 24

early_stopping = EarlyStopping(patience=5,
                               delta=0.001,
                               verbose=True)

start_time = timer()
model_0_results = train(model=model_0,
                optimizer=optimizer,
                loss_fn=loss_fn,
                train_dataloader=train_dataloader,
                validation_dataloader=validation_dataloader,
                early_stopping=early_stopping,
                n_epochs=NUM_EPOCHS)

end_time = timer()

print(f"Total training time: {end_time-start_time:.3f} seconds")
print("MODEL 0 RESULTS : ", model_0_results)

In [ ]:
for key, value in model_0_results.items():
    print(f"{key} : {value}")

### Plot Evaluation Curve

In [ ]:
from utils.util import plot_eval_curves

plot_eval_curves(model_0_results)

### Model is underfitting
- Training accuracy is still low

In [ ]:
# use model_1
from classes.model_1 import Cnn_v1

model_1 = Cnn_v1(input_shape=3,
                 hidden_units=10,
                 output_shape=len(train_dataset.classes)).to(device)

optimizer_1 = torch.optim.Adam(params=model_1.parameters(),
                             lr=0.001)

loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
NUM_EPOCHS = 50

start_time = timer()

model_1_results = train(model=model_1,
                        optimizer=optimizer_1,
                        loss_fn=loss_fn,
                        train_dataloader=train_dataloader,
                        validation_dataloader=validation_dataloader,
                        test_dataloader=test_dataloader,
                        n_epochs=NUM_EPOCHS)

end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

In [ ]:
plot_eval_curves(model_1_results)

In [ ]:
# use data augmentation

data_transform_v1 = v2.Compose(
    [
        v2.Resize(size=(64, 64)),
        v2.RandomHorizontalFlip(p=0.5),
        v2.ToTensor(),
        v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ]
)

train_dataset = ImageFolder(root=root_train_folder_path,
                            transform=data_transform_v1,
                            target_transform=None,  
                            allow_empty=True)

validation_dataset = ImageFolder(root=root_validation_folder_path,
                            transform=data_transform_v1,
                            target_transform=None,
                            allow_empty=True)

test_dataset = ImageFolder(root=root_test_folder_path,
                            transform=data_transform_v1,
                            target_transform=None,
                            allow_empty=True)

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=64,
                              num_workers=1,
                              shuffle=True)

validation_dataloader = DataLoader(dataset=validation_dataset,
                              batch_size=64,
                              num_workers=1,
                              shuffle=False)

test_dataloader = DataLoader(dataset=test_dataset,
                              batch_size=64,
                              num_workers=1,
                              shuffle=False)

In [ ]:
from classes.helpers import EarlyStopping

early_stopping = EarlyStopping(patience=5,
                               delta=0.0001,
                               verbose=True)
model_1 = Cnn_v1(input_shape=3,
                 hidden_units=[32, 64, 128, 256, 512],
                 output_shape=len(train_dataset.classes)).to(device)

optimizer_1 = torch.optim.Adam(params=model_1.parameters(),
                             lr=0.001)

loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
NUM_EPOCHS = 50

start_time = timer()

model_1_results = train(model=model_1,
                        optimizer=optimizer_1,
                        loss_fn=loss_fn,
                        train_dataloader=train_dataloader,
                        validation_dataloader=validation_dataloader,
                        n_epochs=NUM_EPOCHS)

end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")